### Git

In [132]:
from pathlib import Path
from developerscope.analyzer import get_merge_commits_map

current_repo_path = Path().parent.resolve()
# TARGET_REPO = "devQ_testData_PythonProject"
TARGET_REPO = 'toy-python-project'
repo_path = current_repo_path.parent / TARGET_REPO


merge_commits_map, author_mapping = get_merge_commits_map(repo_path)



In [133]:
import git
git_repo = git.Repo(str(repo_path))

all_commits = []
for author, commits in merge_commits_map.items():
    for commit in commits:
        merge_commit = git_repo.commit(commit)
        print(len(all_commits), merge_commit.message.split('\n')[0])
        all_commits.append(merge_commit)
        pass

0 Merge branch 'feature/calc_sub' into 'master'
1 Merge branch 'feature/strings' into 'master'
2 Merge branch 'feature/json-parser' to 'master'
3 Merge branch 'feature/flatten' into 'master'
4 Merge branch 'feature/front-end' into 'master'
5 Merge branch 'feature/api' into 'master'
6 Merge branch 'feature/api-v2' into 'master'


In [134]:
target_commit = all_commits[2]
target_commit.hexsha

'2a909b170dab94861bd113059e6c95fe494621b7'

### Analyzer

In [135]:
SYSTEM_PROMPT = """
You are a secure‑code reviewer.

You will receive:
• the raw `git diff` of a **merge commit**
• the *Halstead total volume* for the changed Python files (objective metric)

Tasks:
1. **Classify** the merge‑request type – choose exactly one from the list.
2. **List potential issues** (security, logic, best practice, etc.) with a severity of LOW‑CRITICAL.
3. If the diff alone is insufficient, call **get_file_contents** with the exact file‑paths you still need.
Return the result strictly as JSON conforming to the MergeRequestAnalysis

OR return nothing and call the `get_file_contents` function. Answer with json only if you think its right thing to do.
Don't shy and request files as much as you want. Try to make only one call, include all interesting files. But it is up to you to make sequential calls"""

SYSTEM_PROMPT_REVIEW = """
You are a senior secure‑code *defender* reviewing an *existing* analysis.

1. **If** you need more context, call `get_file_contents`.
2. **Then** copy the existing analysis but *keep only* issues with severity HIGH
   or CRITICAL.
3. Adjust `effortEstimate` if the filtered list changes the scope.
Output the same `MergeRequestAnalysis` object.
"""

In [136]:
import git

from developerscope.analyzer import get_current_state_paths



def tool_get_file_contents(targer_commit: git.Commit | None = None, files: list[str] = ['example.txt']):
    if targer_commit is not None:
        files = get_current_state_paths(targer_commit)
    return {
    "type": "function",
    "name": "get_file_contents",
    "description": "Function which accepts a list of files in a git repo and produces a their content",
    "strict": True,
    "parameters": {
        "type": "object",
        "required": [
            "files"
        ],
        "properties": {
            "files": {
                "type": "array",
                "description": "List of specific files to read from the git repository",
                "items": {
                    "type": "string",
                    "enum": files,
                    "description": "File name that exists in the git repository"
                }
            }
        },
        "additionalProperties": False
    }
}

In [137]:
from developerscope.analyzer import get_prompt_for_merge_commit


input_messages = [
    {
      "role": "system",
      "content": SYSTEM_PROMPT
    },
    {
      "role": "user",
      "content": get_prompt_for_merge_commit(target_commit)
    }
]

In [138]:
from developerscope._types import MergeRequestAnalysis
import json
from openai import AsyncOpenAI

client = AsyncOpenAI()

with open('schema.json') as file:
    schemaMergeRequest = json.load(file)

async def _get_response(input_messages, tools, required_tool: bool | None):
    if required_tool:
        tool_choice = 'required'
    elif required_tool is None:
        tool_choice = 'none'
    else:
        tool_choice = 'auto'
    return await client.responses.create(
        model="gpt-4.1",
        input=input_messages,
        text={
            "format": schemaMergeRequest
        },
        temperature=0.2,
        tools=tools,
        tool_choice= tool_choice,
        parallel_tool_calls=False
    )
  

In [139]:
from developerscope.analyzer import get_current_state

def call_function(name, args, commit: git.Commit):
    if name == "get_file_contents":
        return get_current_state(commit, args['files'])

In [140]:
from developerscope._types import MergeRequestAnalysis
import json
from openai import AsyncOpenAI



async def run_chat_with_functions(input_messages, tools, required_tool=True):
    max_calls = 3
    for i in range(max_calls):
        if i == max_calls - 1:
            required_tool = None # means forbidden
        response = await(_get_response(input_messages, tools, required_tool=required_tool if tools else False))

        if response.output[0].type == 'message':
            return response.output[0].content[0]
        if response.output[0].type == "function_call":
            tool_call = response.output[0]
            input_messages.append(dict(tool_call))
            name = tool_call.name
            args = json.loads(tool_call.arguments)
            print(name, args)
            result = call_function(name, args, merge_commit)
            input_messages.append({                               
                "type": "function_call_output",
                "call_id": tool_call.call_id,
                "output": str(result)
            })
            required_tool = False

    
    return response.output[0].content[0]


In [141]:
tools = [tool_get_file_contents(target_commit), ]
# tools

In [142]:
response = await run_chat_with_functions(input_messages, tools)
response

get_file_contents {'files': ['README.md', 'unsafe_eval.py']}
README.md
unsafe_eval.py


ResponseOutputText(annotations=[], text='{"hiddenReasoning":"The new function \'json_from_string\' uses Python\'s eval() to parse user-provided strings as JSON, which is extremely dangerous. The eval() function executes arbitrary code, so if a user provides a malicious string, it could lead to arbitrary code execution. This is a critical security vulnerability. The correct approach would be to use the standard library\'s json.loads() for parsing JSON safely. The README does not mention this function, so there is no documentation update or warning about the risk.","type":"Feature","issues":[{"filePath":"unsafe_eval.py","line":"3","issue":"Use of eval() on user input allows arbitrary code execution. This is a critical security vulnerability. Use json.loads() instead for parsing JSON.","level":"CRITICAL"}],"effortEstimate":"Minor"}', type='output_text')

In [143]:
with open('out.json', 'w') as file:
    file.write(response.text)

In [144]:
input_messages = [
    {
      "role": "system",
      "content": SYSTEM_PROMPT_REVIEW
    },
    {
      "role": "user",
      "content": response.text
    }
]

In [145]:
response = await run_chat_with_functions(input_messages, tools)
response

get_file_contents {'files': ['unsafe_eval.py']}
unsafe_eval.py


ResponseOutputText(annotations=[], text='{"hiddenReasoning":"The use of eval() on user input in \'json_from_string\' is a critical security vulnerability, as it allows arbitrary code execution. This should be replaced with a safe alternative like json.loads().","type":"Feature","issues":[{"filePath":"unsafe_eval.py","line":"3","issue":"Use of eval() on user input allows arbitrary code execution. This is a critical security vulnerability. Use json.loads() instead for parsing JSON.","level":"CRITICAL"}],"effortEstimate":"Minor"}', type='output_text')

In [146]:
with open('out_review.json', 'w') as file:
    file.write(response.text)